In [3]:
import numpy as np
import torch
import mmengine
from mmdet3d.structures import LiDARInstance3DBoxes

In [4]:
# info_path = '../data/nuscenes/nuscenes_infos_train.pkl'
# info_path = '../data/CODA/coda_infos_val.pkl'
info_path = '../data/CODA/coda_16lines_infos_val.pkl'
# info_path = '../data/CODA/detection_results.pkl'
info = mmengine.load(info_path)

In [5]:
info['metainfo']

{'categories': {'Car': 0, 'Pedestrian': 1, 'Cyclist': 2},
 'dataset': 'coda',
 'version': 'v1.0',
 'split': 'test'}

In [6]:
len(info['data_list'])

281

In [7]:
info['data_list'][0].keys()

dict_keys(['scene', 'frame', 'sample_idx', 'token', 'timestamp', 'lidar_points', 'images', 'instances', 'ego2global'])

In [8]:
info['data_list'][0]['instances']

[{'bbox_label_3d': 0,
  'bbox_3d': [-0.9067391352685945,
   0.39990630982993025,
   -0.02373945821291379,
   0.5250966241816485,
   0.6964439512911508,
   1.64800068647178,
   -0.014942340926103397],
  'is_occluded': 'Full',
  'instance_id': 'Pedestrian:1'},
 {'bbox_label_3d': 0,
  'bbox_3d': [4.658592347987764,
   -7.354152085573844,
   0.3114288940951262,
   0.9367770422125365,
   0.6303184073941099,
   1.6471728802722168,
   -0.014942293375257097],
  'is_occluded': 'Full',
  'instance_id': 'Pedestrian:8'},
 {'bbox_label_3d': 0,
  'bbox_3d': [-16.605690688790048,
   0.9738310356631175,
   -0.49934294004143176,
   1.0203429086589022,
   0.7004780467827554,
   1.6776112188461318,
   -3.1143894307601827],
  'is_occluded': 'Full',
  'instance_id': 'Pedestrian:4'},
 {'bbox_label_3d': 0,
  'bbox_3d': [13.77538166579805,
   -6.124748555594294,
   0.6261976248052434,
   1.1283675278087222,
   0.783376783056051,
   1.747417494030894,
   -0.018113733833310195],
  'is_occluded': 'None',
  'inst

In [20]:
info.keys()

dict_keys(['lidar_points', 'timestamp', 'lidar_sweeps', 'images', 'box_type_3d', 'box_mode_3d'])

In [22]:
info['box_mode_3d']

0

In [4]:
info.keys()

dict_keys(['sample_idx', 'pre_bboxes', 'pre_labels', 'pre_scores'])

In [ ]:
info['metainfo']['categories']

In [ ]:
class_names = ['Pedestrian', 'Cyclist']
class_labels = [info['metainfo']['categories'][class_name] for class_name in class_names]

In [ ]:
info['pre_bboxes'][10]

In [ ]:
info['metainfo']

In [ ]:
info['data_list'][10]

In [ ]:
info['data_list'][10]['instances']

In [ ]:
info['data_list'][-1]['ego2global']

In [ ]:
len(info['data_list'])

In [ ]:
instance_range = [np.inf, np.inf, np.inf, -np.inf, -np.inf, -np.inf]
corners = []
for sample in info['data_list']:
    bboxes = [instance['bbox_3d'] for instance in sample['instances']]
    bboxes_3d = LiDARInstance3DBoxes(bboxes, origin=(0.5, 0.5, 0.5))
    corner = bboxes_3d.corners.reshape(-1, 3).numpy()
    corners.append(np.concatenate([corner.min(axis=0), corner.max(axis=0)]))
    instance_range[0:3] = np.minimum(instance_range[0:3], np.min(corner, axis=0))
    instance_range[3:6] = np.maximum(instance_range[3:6], np.max(corner, axis=0))

In [ ]:
sum = 0
distribute = [0, 0, 0, 0, 0, 0, 0, 0, 0]
for sample in info['data_list']:    
    for instance in sample['instances']:
        if instance['bbox_label_3d'] == 2:
            sum += 1
            if instance['bbox_3d'][5] < 1.0:
                distribute[0] += 1
            elif instance['bbox_3d'][5] < 1.1:
                distribute[1] += 1
            elif instance['bbox_3d'][5] < 1.2:
                distribute[2] += 1
            elif instance['bbox_3d'][5] < 1.3:
                distribute[3] += 1
            elif instance['bbox_3d'][5] < 1.4:
                distribute[4] += 1
            elif instance['bbox_3d'][5] < 1.5:
                distribute[5] += 1
                print(instance['bbox_3d'], sample['token'])
            elif instance['bbox_3d'][5] < 1.6:
                distribute[6] += 1
            elif instance['bbox_3d'][5] < 1.7:
                distribute[7] += 1
            else:
                distribute[8] += 1
print(sum, distribute)

In [ ]:
corners

In [ ]:
instance_range

In [ ]:
info['data_list'][10]['instances']

In [ ]:
info['data_list'][0]['images']['cam1']

In [ ]:
BBOX_CLASS_TO_ID = {
    # Dynamic Classes
    "Car"                   : 0,
    "Pedestrian"            : 1,
    "Bike"                  : 2,
    "Motorcycle"            : 3,
    "Golf Cart"             : 4, # Unused
    "Truck"                 : 5, # Unused
    "Scooter"               : 6,
    # Static Classes
    "Tree"                  : 7,
    "Traffic Sign"          : 8,
    "Canopy"                : 9,
    "Traffic Light"         : 10,
    "Bike Rack"             : 11,
    "Bollard"               : 12,
    "Construction Barrier"  : 13, # Unused
    "Parking Kiosk"         : 14,
    "Mailbox"               : 15,
    "Fire Hydrant"          : 16,
    # Static Class Mixed
    "Freestanding Plant"    : 17,
    "Pole"                  : 18,
    "Informational Sign"    : 19,
    "Door"                  : 20,
    "Fence"                 : 21,
    "Railing"               : 22,
    "Cone"                  : 23,
    "Chair"                 : 24,
    "Bench"                 : 25,
    "Table"                 : 26,
    "Trash Can"             : 27,
    "Newspaper Dispenser"   : 28,
    # Static Classes Indoor
    "Room Label"            : 29,
    "Stanchion"             : 30,
    "Sanitizer Dispenser"   : 31,
    "Condiment Dispenser"   : 32,
    "Vending Machine"       : 33,
    "Emergency Aid Kit"     : 34,
    "Fire Extinguisher"     : 35,
    "Computer"              : 36,
    "Television"            : 37, # unused
    "Other"                 : 38,
    "Horse"                 : 39,
    # New Classes
    "Pickup Truck"          : 40,
    "Delivery Truck"        : 41,
    "Service Vehicle"       : 42,
    "Utility Vehicle"       : 43,
    "Fire Alarm"            : 44,
    "ATM"                   : 45,
    "Cart"                  : 46,
    "Couch"                 : 47,
    "Traffic Arm"           : 48,
    "Wall Sign"             : 49,
    "Floor Sign"            : 50,
    "Door Switch"           : 51,
    "Emergency Phone"       : 52,
    "Dumpster"              : 53,
    "Vacuum Cleaner"        : 54, # unused
    "Segway"                : 55,
    "Bus"                   : 56,
    "Skateboard"            : 57,
    "Water Fountain"        : 58
}

In [ ]:
METAINFO = {
        'classes':
        tuple(BBOX_CLASS_TO_ID.keys()),
        'version':
        'v1.0-trainval',
        'palette': [
            (255, 158, 0),  # Orange
            (255, 99, 71),  # Tomato
            (255, 140, 0),  # Darkorange
            (255, 127, 80),  # Coral
            (233, 150, 70),  # Darksalmon
            (220, 20, 60),  # Crimson
            (255, 61, 99),  # Red
            (0, 0, 230),  # Blue
            (47, 79, 79),  # Darkslategrey
            (112, 128, 144),  # Slategrey
        ]
    }

In [ ]:
METAINFO['classes']